In [156]:
import pandas as pd
import numpy as np 
from datetime import datetime
import os

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
import json
import re

In [157]:
dates = ['29']
df_fe = pd.DataFrame()
df_cashier = pd.DataFrame()

for date in dates:
    df_fe = pd.concat([df_fe, pd.read_csv('/Users/lap13729/Documents/Data/From Jupyter/Cashier full/Aug{}ZPI_promotionScreen.csv'.format(date))])
    df_cashier = pd.concat([df_cashier, pd.read_csv('/Users/lap13729/Documents/Data/From Jupyter/Cashier_ZDS/Aug/Aug{}_CashierLog.csv'.format(date))])


In [158]:
df_fe.at[0,'metadata']

'{"timeStamp":1661706060871,"os":"ios","osVer":"15.5","user_charge_amount":10000,"promotion_error_code":"","promotion_error_msg":"","app_id":"61","apptransid":"220829000012002"}'

In [159]:
df_cashier = df_cashier[df_cashier['source_type'] == 'ZPI']
df_fe = df_fe[df_fe['eventId'].isin(['02.1104.006','02.1106.006','02.1115.000','02.1104.053'])]
df_fe['metadata'] = df_fe['metadata'].apply(json.loads)

df_cashier['metadata'] = df_cashier['metadata'].apply(json.loads)

In [160]:

list_fe = ['number_of_available_promotion','promotion_error_code']
for i in list_fe:
    df_fe[i] = df_fe['metadata'].map(lambda x: x[i] if i in x else np.nan)

list_cashier = ['total_promotion','best_promotion_applied']
for j in list_cashier:
    df_cashier[j] = df_cashier['metadata'].map(lambda x: x[j] if j in x else np.nan)

In [161]:
viewPromoList = df_fe[
    df_fe['eventId'].isin(['02.1104.006','02.1106.006'])
]

applyPromo = df_fe[
    df_fe['eventId'] == '02.1115.000'
]

viewPromoDetail = df_fe[
    df_fe['eventId'] == '02.1104.053'
] 

temp_cashier = df_cashier[
    (df_cashier['source_type'] == 'ZPI')
    & (~df_cashier['domain'].isin(['Funding Channel','IBFT','Transfer','Withdraw']))
]

In [162]:
viewPromoList.head()

,clientIp,eventId,metadata,preUrlPath,previousEventId,timeStamp,trackingSessionId,urlPath,zaloId,zaloPayId,screenId,number_of_available_promotion,promotion_error_code
8,42.113.204.122,02.1106.006,"{'timeStamp': 1661706067630, 'os': 'android', ...",https://social.zalopay.vn/spa/v2/telco/topup,02.1101.006,1661706067718,ed1f167b-b2de-413e-879b-cb3e0b8c75a5,https://social.zalopay.vn/spa/v2/telco/topup,7.535615e+18,200805000066668,1106,1.0,NaN
14,1.52.37.71,02.1104.006,"{'timeStamp': 1661706071114, 'os': 'ios', 'osV...",https://social.zalopay.vn/spa/v2/telco/topup,02.1103.006,1661706071309,b4e518d9-e00e-42a0-8733-ed724a27ac55,https://social.zalopay.vn/spa/v2/telco/topup,8.208762e+18,220209004500014,1104,0.0,NaN
16,116.98.161.193,02.1104.006,"{'timeStamp': 1661706070920, 'os': 'ios', 'osV...",https://social.zalopay.vn/spa/v2/telco/topup,02.1103.006,1661706071077,0f57e506-0362-490c-8284-890cc0509169,https://social.zalopay.vn/spa/v2/telco/topup,6.058261e+18,200828000019118,1104,1.0,NaN
19,113.185.42.7,02.1104.006,"{'timeStamp': 1661706074599, 'os': 'ios', 'osV...",https://social.zalopay.vn/spa/v2/telco/topup,02.1103.006,1661706074742,413ad605-0e9f-4cbc-8040-87938942f800,https://social.zalopay.vn/spa/v2/telco/topup,7.567310e+18,220812004000678,1104,0.0,NaN
23,222.253.140.238,02.1104.006,"{'timeStamp': 1661706076873, 'os': 'ios', 'osV...",https://social.zalopay.vn/spa/v2/merchant?sour...,02.1103.006,1661706076996,beb938a9-f53b-426f-bccb-46d0392002a4,https://social.zalopay.vn/spa/v2/merchant?sour...,4.768978e+18,210131003015460,1104,0.0,NaN


In [163]:
#Filter View Promo or not?
temp_cashier['viewPromo?'] = 0
temp_cashier['viewPromo?'][
    (temp_cashier['trackingSessionId'].isin(viewPromoList['trackingSessionId'].unique().tolist()))
] = 1


In [164]:
#Filter have valid promo when open list or not
haveValid = viewPromoList[viewPromoList['number_of_available_promotion'].astype('int') > 0]

temp_cashier['haveValid?'] = 0
temp_cashier['haveValid?'][
    temp_cashier['trackingSessionId'].isin(haveValid['trackingSessionId'].unique().tolist())
] = 1


In [165]:
#Filter apply promo or not
temp_cashier['applyPromo?'] = 0
temp_cashier['applyPromo?'][
    temp_cashier['trackingSessionId'].isin(applyPromo['trackingSessionId'].unique().tolist())
] = 1

In [166]:
#Filter apply promo success or not
applySuccess = applyPromo[applyPromo['promotion_error_code'] == '']
temp_cashier['applySuccess?'] = 0
temp_cashier['applySuccess?'][
    temp_cashier['trackingSessionId'].isin(applySuccess['trackingSessionId'].unique().tolist())
] = 1

In [167]:
#Filter view promotion detail
temp_cashier['viewDetail?'] = 0
temp_cashier['viewDetail?'][
    temp_cashier['trackingSessionId'].isin(viewPromoDetail['trackingSessionId'].unique().tolist())
] = 1

In [168]:
temp_cashier.head(3)

,appName,app_id,description,event_id,funnel_steps,key_id,metadata,orderamount,orderamountwithfee,previousEventId,source_type,timeStamp,timeStamp2,trackingSessionId,user_id,ymd_etl,zp_trans_id,user_id2,orderSource,pmcID,transStatus,transID,bcCode,is_topup,total_fee,valid_bank_linking_count,total_promotion,selected_sof,metadata_sof,is_napas,is_fpu,is_npu,is_rpu,user_type,integrated_solution,sof,is_submitted,is_success,is_newcashier,k_before,k_after,os_zpa,domain,userAgent,os_zpi,os,po_trans_type,best_promotion_applied,viewPromo?,haveValid?,applyPromo?,applySuccess?,viewDetail?
2,EVN HCM,372.0,1. Cashier Load,02.1101.000,1. Cashier Load,NaN,"{'timeStamp': 1661758781450, 'os': 'android', ...",885686.0,885686.0,02.1112.000,ZPI,1661758782865,1661758781450,002372e4-d955-41b9-9db9-60eede6316e9,200221000037984,2022-08-29 14:39:42.865,NaN,200221000037984,21.0,38.0,1.0,2.208290e+14,0.0,1.0,0.0,1.0,0,Bank,Wallet_38_status_100Agribank_970405_3844_39_st...,0,0,0,1,R,Ko xac dinh,Wallet Balance,1,1,0,2,2,undefined,Billing,Mozilla/5.0 (Linux; Android 10; SM-M515F Build...,android,android,0,None,0,0,0,0,0
4,Lazada.vn,1215.0,1. Cashier Load,02.1103.000,1. Cashier Load,NaN,"{'timeStamp': 1661760106346, 'os': 'ios', 'osV...",1000.0,1000.0,02.1112.000,ZPI,1661760106598,1661760106346,00566387-12e0-45f6-8b72-96e1c0302903,200211000053694,2022-08-29 15:01:46.598,NaN,200211000053694,21.0,38.0,1.0,2.208290e+14,0.0,NaN,0.0,1.0,2,Wallet,Wallet_38_status_1Vietinbank_970415_7650_39_st...,0,0,0,1,R,Ko xac dinh,Wallet Balance,1,1,0,3,3,undefined,QR/App2App/Web2App,Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like M...,ios,ios,0,True,0,0,0,0,0
5,Thẻ Điện Thoại,12.0,1. Cashier Load,02.1103.000,1. Cashier Load,NaN,"{'timeStamp': 1661755385232, 'os': 'ios', 'osV...",10000.0,10000.0,02.1103.033,ZPI,1661755385463,1661755385232,009378ea-a8b6-4d17-a50c-04ad7a01f064,220829004502034,2022-08-29 13:43:05.463,NaN,220829004502034,20.0,38.0,-332.0,2.208290e+14,0.0,NaN,0.0,1.0,0,Wallet,Wallet_38_status_1Visa Card_408904_6429_41_sta...,0,0,1,0,N,Ko xac dinh,Wallet Balance,1,0,0,2,2,undefined,Telco,Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like M...,ios,ios,1,None,0,0,0,0,0


Done Filtering! :)

---

Graph (1)

In [149]:
dr = ['Load Cashier','Have Promo','Not Have Promo','Auto-applied','Not Auto-applied','Have Promo - View Promo','Have Promo - Not View Promo','Not Have Promo - View Promo','Not Have Promo - Not View Promo','Have Valid Promo in List','Not Have Valid Promo in List','Apply Promo','Not Apply Promo','Apply Success','Apply Failed','Not Apply - View Detail','Not Apply - Not View Detail','Not Have Valid Promo in List - View Detail','Not Have Valid Promo in List - Not View Detail']
dr = pd.DataFrame(dr)
dr

,0
0,Load Cashier
1,Have Promo
2,Not Have Promo
3,Auto-applied
4,Not Auto-applied
5,Have Promo - View Promo
6,Have Promo - Not View Promo
7,Not Have Promo - View Promo
8,Not Have Promo - Not View Promo
9,Have Valid Promo in List


In [150]:
total_loads = temp_cashier.shape[0]

In [151]:
df = temp_cashier

In [154]:
#Ko co best promotion applied false ma click promo ... nen phai chia lai dimension thoi 

True     47974
False       14
Name: best_promotion_applied, dtype: int64

In [152]:
res = pd.DataFrame(
    columns = ['name','Load Cashier','Have Promo','Not Have Promo','Auto-applied','Not Auto-applied','Have Promo - View Promo','Have Promo - Not View Promo','Not Have Promo - View Promo','Not Have Promo - Not View Promo','Have Valid Promo in List','Not Have Valid Promo in List','Apply Promo','Not Apply Promo','Apply Success','Apply Failed','Not Apply - View Detail','Not Apply - Not View Detail','Not Have Valid Promo in List - View Detail','Not Have Valid Promo in List - Not View Detail']
)
res['name'] = ['load','submit','success','distribution','submission rate','be sr','e2e sr']

for i, column in enumerate(['Load Cashier','Have Promo','Not Have Promo','Auto-applied','Not Auto-applied','Have Promo - View Promo','Have Promo - Not View Promo','Not Have Promo - View Promo','Not Have Promo - Not View Promo','Have Valid Promo in List','Not Have Valid Promo in List','Apply Promo','Not Apply Promo','Apply Success','Apply Failed','Not Apply - View Detail','Not Apply - Not View Detail','Not Have Valid Promo in List - View Detail','Not Have Valid Promo in List - Not View Detail']):
    if i == 0:
        df = temp_cashier
    elif i == 1:
        df = temp_cashier[
            temp_cashier['total_promotion'].astype('int') >0
        ]
    elif i == 2:
        df = temp_cashier[
            temp_cashier['total_promotion'].astype('int') == 0
        ]
    elif i == 3:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] == True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
        ]
    elif i == 4:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)

        ]
    elif i == 5:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') >0)
            & (temp_cashier['viewPromo?'] == 1)
        ]
    elif i == 6:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') >0)
            & (temp_cashier['viewPromo?'] == 0)
        ]
    elif i == 7:
        df = temp_cashier[
            (temp_cashier['total_promotion'].astype('int') == 0)
            & (temp_cashier['viewPromo?'] == 1)
        ]
    elif i == 8:
        df = temp_cashier[
            (temp_cashier['total_promotion'].astype('int') == 0)
            & (temp_cashier['viewPromo?'] == 0)
        ]
    elif i == 9:
        df = temp_cashier[
            (temp_cashier['total_promotion'].astype('int') >0)
            & (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
        ]
    elif i == 10:
        df = temp_cashier[
            (temp_cashier['total_promotion'].astype('int') >0)
            & (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 0)
        ]
    elif i == 11:
        df = temp_cashier[
            (temp_cashier['total_promotion'].astype('int') >0)
            & (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] ==1)
        ]
    elif i == 12:
        df = temp_cashier[
            (temp_cashier['total_promotion'].astype('int') >0)
            & (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] ==0)
        ]
    elif i == 13:
        df = temp_cashier[
            (temp_cashier['total_promotion'].astype('int') >0)
            & (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] ==1)
            & (temp_cashier['applySuccess?'] ==1)
        ]
    elif i == 14:
        df = temp_cashier[
            (temp_cashier['total_promotion'].astype('int') >0)
            & (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] ==1)
            & (temp_cashier['applySuccess?'] ==0)
        ]
    elif i == 15:
        df = temp_cashier[
            (temp_cashier['total_promotion'].astype('int') >0)
            & (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] ==0)
            & (temp_cashier['viewDetail?'] ==1)
        ]
    elif i == 16:
        df = temp_cashier[
            (temp_cashier['total_promotion'].astype('int') >0)
            & (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] ==0)
            & (temp_cashier['viewDetail?'] ==0)
        ]
    elif i == 17:
        df = temp_cashier[
            (temp_cashier['total_promotion'].astype('int') >0)
            & (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 0)
            & (temp_cashier['viewDetail?'] == 1)
        ]
    elif i == 18:
        df = temp_cashier[
            (temp_cashier['total_promotion'].astype('int') >0)
            & (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 0)
            & (temp_cashier['viewDetail?'] == 0)
        ]
    
    #index
    load = len(df)
    submit = len(df[df['is_submitted'] == 1])
    success = len(df[df['is_success'] == 1])
    distribution = load/total_loads
    submission_rate = submit/load
    be_sr = success/submit
    e2e_sr = success/load

    #table
    res[column] =[
        load,
        submit,
        success,
        distribution,
        submission_rate,
        be_sr,
        e2e_sr,
    ]

#dung roi, chi chia lai dimension thoi 


ZeroDivisionError: division by zero

In [ ]:
res

,name,Load Cashier,Have Promo,Not Have Promo,Auto-applied,Not Auto-applied,Have Promo - View Promo,Have Promo - Not View Promo,Not Have Promo - View Promo,Not Have Promo - Not View Promo,Have Valid Promo in List,Not Have Valid Promo in List,Apply Promo,Not Apply Promo,Apply Success,Apply Failed,Not Apply - View Detail,Not Apply - Not View Detail,Not Have Valid Promo in List - View Detail,Not Have Valid Promo in List - Not View Detail
0,load,240263.000000,90010.000000,150253.000000,47918.000000,192345.000000,22110.000000,19982.000000,38336.000000,111917.000000,15979.000000,6131.000000,14391.000000,1588.000000,8637.000000,5754.000000,97.000000,1491.000000,432.000000,5699.000000
1,submit,147048.000000,43440.000000,103608.000000,27316.000000,119732.000000,6128.000000,9996.000000,13007.000000,90601.000000,4658.000000,1470.000000,4394.000000,264.000000,3786.000000,608.000000,7.000000,257.000000,84.000000,1386.000000
2,success,137666.000000,39181.000000,98485.000000,25285.000000,112381.000000,5306.000000,8590.000000,12070.000000,86415.000000,4045.000000,1261.000000,3832.000000,213.000000,3547.000000,285.000000,7.000000,206.000000,70.000000,1191.000000
3,distribution,1.000000,0.374631,0.625369,0.199440,0.800560,0.092024,0.083167,0.159558,0.465810,0.066506,0.025518,0.059897,0.006609,0.035948,0.023949,0.000404,0.006206,0.001798,0.023720
4,submission rate,0.612029,0.482613,0.689557,0.570057,0.622486,0.277160,0.500250,0.339289,0.809537,0.291508,0.239765,0.305330,0.166247,0.438347,0.105666,0.072165,0.172368,0.194444,0.243201
5,be sr,0.936198,0.901957,0.950554,0.925648,0.938605,0.865862,0.859344,0.927962,0.953797,0.868398,0.857823,0.872098,0.806818,0.936873,0.468750,1.000000,0.801556,0.833333,0.859307
6,e2e sr,0.572980,0.435296,0.655461,0.527672,0.584268,0.239982,0.429887,0.314848,0.772135,0.253145,0.205676,0.266278,0.134131,0.410675,0.049531,0.072165,0.138162,0.162037,0.208984


In [148]:
res.loc[3,'Auto-applied'] + res.loc[3, 'Not Auto-applied']

1.0

---

Graph (2)

In [176]:
dr = ['Load Cashier',
    'Not Auto-applied',
    'Auto-applied',
    'Not Auto-applied - Have Promo',
    'Not Auto-applied - Not Have Promo',
    'Auto-applied - View Promo List',
    'Auto-applied - Not View Promo List',
    'Not Auto-applied - Have Promo - Not View Promo List',
    'Not Auto-applied - Have Promo - View Promo List',
    'Not Auto-applied - Not Have Promo - Not View Promo List',
    'Not Auto-applied - Not Have Promo - View Promo List',
    'Have Valid Promo in List',
    'Not Have Valid Promo in List',
    'Apply Promo',
    'Not Apply Promo',
    'Apply Success',
    'Apply Failed',
    'Not Apply - View Detail',
    'Not Apply - Not View Detail',
    'Not Have Valid Promo in List - View Detail',
    'Not Have Valid Promo in List - Not View Detail',
    ]

dr = pd.DataFrame(dr)
dr

,0
0,Load Cashier
1,Not Auto-applied
2,Auto-applied
3,Not Auto-applied - Have Promo
4,Not Auto-applied - Not Have Promo
5,Auto-applied - View Promo List
6,Auto-applied - Not View Promo List
7,Not Auto-applied - Have Promo - Not View Promo...
8,Not Auto-applied - Have Promo - View Promo List
9,Not Auto-applied - Not Have Promo - Not View P...


In [169]:
total_loads = temp_cashier.shape[0]

In [170]:
df = temp_cashier

In [171]:
df['best_promotion_applied'].unique()

array([None, True, False], dtype=object)

In [182]:
res = pd.DataFrame(
    columns = ['name',
                'Load Cashier',
                'Not Auto-applied',
                'Auto-applied',
                'Not Auto-applied - Have Promo',
                'Not Auto-applied - Not Have Promo',
                'Auto-applied - View Promo List',
                'Auto-applied - Not View Promo List',
                'Not Auto-applied - Have Promo - Not View Promo List',
                'Not Auto-applied - Have Promo - View Promo List',
                'Not Auto-applied - Not Have Promo - Not View Promo List',
                'Not Auto-applied - Not Have Promo - View Promo List',
                'Have Valid Promo in List',
                'Not Have Valid Promo in List',
                'Apply Promo',
                'Not Apply Promo',
                'Apply Success',
                'Apply Failed',
                'Not Apply - View Detail',
                'Not Apply - Not View Detail',
                'Not Have Valid Promo in List - View Detail',
                'Not Have Valid Promo in List - Not View Detail',
                ]
)
res['name'] = ['load','submit','success','distribution','submission rate','be sr','e2e sr']

for i, column in enumerate(['Load Cashier',
                            'Not Auto-applied',
                            'Auto-applied',
                            'Not Auto-applied - Have Promo',
                            'Not Auto-applied - Not Have Promo',
                            'Auto-applied - View Promo List',
                            'Auto-applied - Not View Promo List',
                            'Not Auto-applied - Have Promo - Not View Promo List',
                            'Not Auto-applied - Have Promo - View Promo List',
                            'Not Auto-applied - Not Have Promo - Not View Promo List',
                            'Not Auto-applied - Not Have Promo - View Promo List',
                            'Have Valid Promo in List',
                            'Not Have Valid Promo in List',
                            'Apply Promo',
                            'Not Apply Promo',
                            'Apply Success',
                            'Apply Failed',
                            'Not Apply - View Detail',
                            'Not Apply - Not View Detail',
                            'Not Have Valid Promo in List - View Detail',
                            'Not Have Valid Promo in List - Not View Detail',
                            ]):
    if i == 0:
        df = temp_cashier
    elif i == 1:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
        ]
    elif i == 2:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] == True)
        ]
    elif i == 3:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
        ]
    elif i == 4:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') == 0)

        ]
    elif i == 5:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] == True)
            & (temp_cashier['viewPromo?'] == 1)
        ]
    elif i == 6:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] == True)
            & (temp_cashier['viewPromo?'] == 0)
        ]
    elif i == 7:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 0)
        ]
    elif i == 8:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
        ]
    elif i == 9:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') == 0)
            & (temp_cashier['viewPromo?'] == 0)
        ]
    elif i == 10:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') == 0)
            & (temp_cashier['viewPromo?'] == 1)
        ]
    elif i == 11:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
        ]
    elif i == 12:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 0)
        ]
    elif i == 13:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] == 1)
        ]
    elif i == 14:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] == 0)
        ]
    elif i == 15:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] == 1)
            & (temp_cashier['applySuccess?'] == 1)
        ]
    elif i == 16:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] == 1)
            & (temp_cashier['applySuccess?'] == 0)
        ]
    elif i == 17:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] == 0)
            & (temp_cashier['viewDetail?'] == 1)
        ]
    elif i == 18:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] == 0)
            & (temp_cashier['viewDetail?'] == 0)
        ]
    elif i == 19:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 0)
            & (temp_cashier['viewDetail?'] == 1)
        ]
    elif i == 20:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 0)
            & (temp_cashier['viewDetail?'] == 0)
        ]
    
    #index
    load = len(df)
    submit = len(df[df['is_submitted'] == 1])
    success = len(df[df['is_success'] == 1])
    distribution = load/total_loads
    submission_rate = submit/load
    be_sr = success/submit
    e2e_sr = success/load

    #table
    res[column] =[
        load,
        submit,
        success,
        distribution,
        submission_rate,
        be_sr,
        e2e_sr,
    ]



In [183]:
res

,name,Load Cashier,Not Auto-applied,Auto-applied,Not Auto-applied - Have Promo,Not Auto-applied - Not Have Promo,Auto-applied - View Promo List,Auto-applied - Not View Promo List,Not Auto-applied - Have Promo - Not View Promo List,Not Auto-applied - Have Promo - View Promo List,Not Auto-applied - Not Have Promo - Not View Promo List,Not Auto-applied - Not Have Promo - View Promo List,Have Valid Promo in List,Not Have Valid Promo in List,Apply Promo,Not Apply Promo,Apply Success,Apply Failed,Not Apply - View Detail,Not Apply - Not View Detail,Not Have Valid Promo in List - View Detail,Not Have Valid Promo in List - Not View Detail
0,load,240263.000000,192289.000000,47974.000000,42092.000000,150197.000000,8330.000000,39644.000000,19982.000000,22110.000000,111865.000000,38332.000000,15979.000000,6131.000000,14391.000000,1588.000000,8637.000000,5754.000000,97.000000,1491.000000,432.000000,5699.000000
1,submit,147048.000000,119697.000000,27351.000000,16124.000000,103573.000000,3138.000000,24213.000000,9996.000000,6128.000000,90567.000000,13006.000000,4658.000000,1470.000000,4394.000000,264.000000,3786.000000,608.000000,7.000000,257.000000,84.000000,1386.000000
2,success,137666.000000,112350.000000,25316.000000,13896.000000,98454.000000,2768.000000,22548.000000,8590.000000,5306.000000,86385.000000,12069.000000,4045.000000,1261.000000,3832.000000,213.000000,3547.000000,285.000000,7.000000,206.000000,70.000000,1191.000000
3,distribution,1.000000,0.800327,0.199673,0.175191,0.625136,0.034670,0.165003,0.083167,0.092024,0.465594,0.159542,0.066506,0.025518,0.059897,0.006609,0.035948,0.023949,0.000404,0.006206,0.001798,0.023720
4,submission rate,0.612029,0.622485,0.570121,0.383066,0.689581,0.376711,0.610761,0.500250,0.277160,0.809610,0.339299,0.291508,0.239765,0.305330,0.166247,0.438347,0.105666,0.072165,0.172368,0.194444,0.243201
5,be sr,0.936198,0.938620,0.925597,0.861821,0.950576,0.882091,0.931235,0.859344,0.865862,0.953824,0.927956,0.868398,0.857823,0.872098,0.806818,0.936873,0.468750,1.000000,0.801556,0.833333,0.859307
6,e2e sr,0.572980,0.584277,0.527703,0.330134,0.655499,0.332293,0.568762,0.429887,0.239982,0.772225,0.314854,0.253145,0.205676,0.266278,0.134131,0.410675,0.049531,0.072165,0.138162,0.162037,0.208984


---